# **Basics of Mobile Robotics Project** 
## **EPFL - December 2023**
### **Professor: Francesco Mondada**
### **Authors:**
-Angelo Giovine

-Nome2

-Nome3

-Nome4

## **Tables of contents**

- [Introduction](#Introduction)
  - [Modules and libraries required](#Modules-and-libraries-required)
  - [Environment](#Environment)
- [Vision](#capitolo-2)
  - [Sezione 2.1](#sezione-2.1)
- [Global Navigation](#capitolo-2)
  - [Sezione 2.1](#sezione-2.1)
- [Local Navigation](#capitolo-2)
  - [Sezione 2.1](#sezione-2.1)
- [Filtering](#Filtering:-Extended-Kalman-Filter)
  - [Why Kalman?](#Deriving-the-state-space-model-of-the-Thymio-Robot)
  - [Deriving the state space model of the Thymio Robot](#Deriving-the-state-space-model-of-the-Thymio-Robot)
  - [Dealing with the input](#Dealing-with-the-input)
  - [Experimentation to derive $Cl$ and $Cr$](#Experimentation-to-derive-$Cl$-and-$Cr$)
  - [Calcultating the covariance matrix of the motion model $Q_t$](#Calcultating-the-covariance-matrix-of-the-motion-model-$Q_t$)
  - [Deriving the observation model of the Thymio Robot](#Deriving-the-observation-model-of-the-Thymio-Robot)
  - [Calcultating the covariance matrix of the observation model $R_t$ ](#Calcultating-the-covariance-matrix-of-the-observation-model-$R_t$ )
  - [testImplementing the Extended Kalman Filter](#Implementing-the-Extended-Kalman-Filter)
  - [Sezione 2.1](#sezione-2.1)
- [Control law ](#Control-law)
- [Main](#Main)
- [Conclusions ](#capitolo-2)
  - [Sources](#sezione-2.1)



# **Demo videos**

The optimal path is displayed in white, while the real-time estimated position of the Thymio, obtained through filtering, is indicated by a black dotted line.

Full demo + Noraml situation + Kidnapping + Camera obstructed + Local obstacles: https://youtu.be/rz_EOHwGMOw

Full demo: https://youtu.be/DTvLO2GhbcM
    
Normal situation: https://youtu.be/M8M3lAdUMPQ

Kidnapping: https://youtu.be/5M4zRgAHsiA

Camera obstructed: https://youtu.be/vcZKryRRfx0

Local obstacles: https://youtu.be/Q0z2UTtgOtY


# **Introduction**

The project is structured around five distinct modules, each tailored to a specific aspect of the Thymio Robot's navigation and operational capabilities:

##### Vision Module: 
This component is responsible for capturing and processing visual data, primarily for map generation and the robot's localization within the map.

##### Global Navigation Module: 
It focuses on plotting a global path for the robot. This is achieved through the implementation of the Dijkstra algorithm on a visibility graph, enabling the robot to plan its route from its current position to the designated goal.

##### Local Navigation Module: 
This module is designed for real-time navigation, particularly in dealing with immediate obstacles. It facilitates local obstacle avoidance and dynamically adjusts the robot's path as new obstacles are detected.

##### Filtering Module: 
The core function of this module is continuous state estimation, primarily achieved through the use of an Extended Kalman Filter. This ensures that the robot has an accurate understanding of its state in relation to the environment, even when the camera is unable to capture visual data.

##### Control Law Module: 
This final module governs the robot's movement and behavior. It includes algorithms for path following and control laws to ensure that the robot adheres to the planned path, and it responds appropriately to changes in the environment, such as being moved unexpectedly (kidnapping detection) or encountering unforeseen obstacles.




#### **Modules and libraries required:**

-------> INSERIRE TUTTE LE LIBRERE UTILIZZATE E I MODULI, CON SCRIPT PER INSTALLAZIONE

#### **Environment**

For the chosen environment, we have selected a monochromatic blue background, complemented by 2D green obstacles and a yellow target area. The Thymio robot is identified by a distinctive red "mask" applied to it.

# INSERIRE QUI FOTO AMBIENTE 
# INSEIRE QUI FOTO THYMIO


In the development of our Mobile Robotics Project, we employed a forward-thinking strategy by incorporating ChatGPT-4's capabilities. Our choice was influenced by ChatGPT-4's sophisticated computational prowess and its adeptness in crafting intricate geometric figures. We primarily utilized the model to construct a variety of polygons, which functioned as fixed obstacles on our map. We observed that while the model could produce a broad spectrum of shapes, from simple triangles to elaborate polygons, it encountered difficulties in creating a PDF with non-overlapping shapes. Additionally, it was challenged in randomly generating polygons, often resulting in blank outputs or error messages. Consequently, we intervened by specifying the dimensions, forms, and hues of the polygons. As a result, our map was not entirely AI-generated but rather a collaborative effort, blending our specific polygon choices with GPT-4's assembly skills. This experience once again underscored the limitations of a Large Language Model (LLM) in tasks demanding significant creativity.

<div align="center">
    <img src="img/GPT_error.png" alt="GPT error message" width="450" style="margin-right: 10px;"/>
    <img src="img/GPT_env.png" alt="polygons generated by GPT4 " width="550"/>
</div>

# **Vision**

#### Sezione 2.1

<div align="center">
    <img src="img/iPhone.jpg" alt="very bad image of the concept" width="400"/>
</div>

# **Global Navigation**

#### Sezione 2.1

# **Local Navigation**

#### Sezione 2.1

# **Filtering: Extended Kalman Filter**

#### **Why Kalman?**

During the course we have encountered different Bayes filter to estimate the state of a robot in a given environment. For example, we could have used a particle filter, but it is more computationally expensive and it is not necessary for this application. For our task we have choosen the Kalman filter, a recursive filter that estimates the state of a system from a series of noisy measurements. It is a very efficient algorithm that can be used to solve the localization problem of a robot, but has a crucial assumption about the distribution of the noise. In fact, it assumes that the noise is Gaussian distributed, which is not always the case in real life. However, in differential robots, like the Thymio, odometry is primarily based on the velocity sensors of the motors to calculate the robot's position and orientation ($x, y, \gamma$). The errors in odometry can arise from various sources:
- Sensor Measurement Errors: The sensors might not be perfectly accurate or might exhibit some drift over time.
- Mechanical Misalignments: Slight imperfections in the robot's construction or wear in components can cause discrepancies.
- Differential Motion Transmission Variances: Factors like friction, tire pressure, or differences in the movement surface can affect how motion is transmitted from the motors to the wheels.
- Slippage Error: The wheels may slip or lose traction, leading to inaccurate distance measurements.

Since each of these error sources contributes small, independent increments to the overall odometry error. According to the Central Limit Theorem, when a large number of small, independent effects are summed, their overall distribution tends to approach a Gaussian distribution. This implies that even if each individual source of error might not be Gaussian, their cumulative effect tends towards a normal distribution. 

Since the odometry noise can be assumed to be Gaussian distributed, the Kalman filter, that's easy to implement, is a good choice for our application.

Once again we have decied to collaborate with LLM. This time we wanted to generate an image suitable for this paragraph, but even if we were precise about the contest ChatGPT-4 was able to produce an image not appropriated for this paragraph, the image is not scientific or accurated but we decided to use it anyway, to highlight another limit of LLM.

<div align="center">
    <img src="img/GPT_Kalman.png" alt="very bad image of the concept" width="400"/>
</div>


#### **Deriving the state space model of the Thymio Robot**

Source: https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/ 

<div align="center">
    <img src="img/state_space_.png" alt="2D image of a differential drive robot" width="600"/>
</div>


First of all, we need to define the state and input variables. We will use the following state variables: $x$ and $y$ for the position, and $\gamma$ for the orientation, since we can control our Thyimio in speed, we can define our input variables $v$ forward velocity, and $\omega$ angular velocity . After some simple trigonometric analysis is easy to define the state vector $s$ is defined as follows: 
$$
s_{t} =
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
=
\begin{bmatrix}
x_{t-1} + v_{t-1}\cos\gamma_{t-1} \cdot \Delta t \\
y_{t-1} + v_{t-1}\sin\gamma_{t-1} \cdot \Delta t \\
\gamma_{t-1} + \omega_{t-1} \cdot \Delta t
\end{bmatrix}
=
\begin{bmatrix}
f_1 \\
f_2 \\
f_3
\end{bmatrix}
$$

Hence our input vector $u$ is defined as follows:

$$
u_{t} = 
\begin{bmatrix}
    v_{t} \\ 
     \omega_{t}
\end{bmatrix}
$$
We would now define our state space model as our system is linaer so in this form:  $s_{t}= A_{t-1}s_{t-1} + B_{t-1}u_{t-1}$, where $A_{t-1}$ is the state matrix and $B_{t-1}$ is the input matrix. But our original system is not linear. However, we can approximate our system by using the Jacobian matrix.
Our Thymio moves only when it receives instructions to rotate its wheels. As a result, in this scenario, the $A_{t-1}$ matrix is an identity matrix (linear relation). For the $B_{t-1}$ matrix we can use the Jacobian matrix, which is defined as follows:

$$
B_{t-1} = 
\begin{bmatrix}
\frac{\partial f_1}{\partial v_{t-1}} & \frac{\partial f_1}{\partial \omega_{t-1}} \\
\frac{\partial f_2}{\partial v_{t-1}} & \frac{\partial f_2}{\partial \omega_{t-1}} \\
\frac{\partial f_3}{\partial v_{t-1}} & \frac{\partial f_3}{\partial \omega_{t-1}}
\end{bmatrix}
=
\begin{bmatrix}
\cos(\gamma_{t-1} \cdot \Delta t) & 0 \\
\sin(\gamma_{t-1} \cdot \Delta t) & 0 \\
0 & \Delta t
\end{bmatrix}
$$
If we consider also the process noise $ w_{t} = [ w^1_{t}, w^2_{t}, w^3_{t} ] $
 , we can define our state space model as follows:
$$s_{t}= A_{t-1}s_{t-1} + B_{t-1}u_{t-1} +w_{t-1}$$
$$
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_{t-1} \\
y_{t-1} \\
\gamma_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
\cos(\gamma_{t-1} \cdot \Delta t) & 0 \\
\sin(\gamma_{t-1} \cdot \Delta t) & 0 \\
0 & \Delta t
\end{bmatrix}
\begin{bmatrix}
v_{t-1} \\
\omega_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
w^1_{t-1} \\
w^2_{t-1} \\
w^3_{t-1}
\end{bmatrix}
$$


Note that we are assuming that the process noise is zero-mean Gaussian noise with covariance matrix $Q_k$, which is computed after some experimentation explained in the next section.



#### **Dealing with the inputs** 

Source: https://www.youtube.com/watch?v=aE7RQNhwnPQ

If we notice, our state space model is controlled in $v$ forward velocity, and $\omega$ angular velocity but in reality we can only control the speed of the two motors $v_{right}$, and $v_{left}$ of our Thymio. So we need to find a way to convert the speed of the robot $v$ and $\omega$ into $v_{right}$, and $v_{left}$. We can do this by using the following equations:

$v_{right} = \frac{2v+\omega L}{2R}$ , $v_{left} = \frac{2v-\omega L}{2R}$

Where $L$ is the distance between the two wheels and $R$ is the radius of the wheels.

In order to control the Thymio we must convert the speed of the two motors from the Thymio unit system to $v_{right}$, and $v_{left}$  we can do this buy using the convertion factor $Cr$ and $Cl$ calculated through experimentation.




In [2]:
def speed_converison(v,omega,R,L,Cr,Cl):
    # v: [mm/s] input variable 1
    # omega: [rad/s] input variable 2
    # R: [mm] wheel radius of the Thymio
    # L: [mm] distance between the wheels of the Thymio
    # Cr: [(s/rad)*Thymio units] convertion factor from rad/s to Thymio speed unit
    # Cl: [(s/rad)*Thymio units] convertion factor from rad/s to Thymio speed unit
    # return: [Thymio unit, Thymio unit]
    v_r = ((2*v + omega*L)/(2*R))*Cr
    v_l = ((2*v - omega*L)/(2*R))*Cl
    return v_r, v_l

In [6]:
def inverse_speed_conversion(v_r, v_l, R, L, Cr, Cl):
    # v_r: [Thymio unit] input variable 1
    # v_l: [Thymio unit] input variable 2
    # R: [mm] wheel radius of the Thymio
    # L: [mm] distance between the wheels of the Thymio
    # Cr: [(s/rad)*Thymio units] convertion factor from rad/s to Thymio speed unit
    # Cl: [(s/rad)*Thymio units] convertion factor from rad/s to Thymio speed unit
    v = (R/2)*(v_r/Cl + v_l/Cr)
    omega = (R/L)*(v_r/Cl - v_l/Cr)
    return v, omega


#### **Experimentation to derive $Cl$ and $Cr$**

In order to use the $Cl$ and $Cr$ values in the program, we need to derive them from the experimental data. This section contains the collected data and the results. First we have marked each wheel of the Thymio robot then we have run the robot at a fixed speed $spd = 200$  $Thymio units$ and measured the number of revolution of each wheel in a fixed time interval $t$. We have repeated this experiment 10 times and calculated our average values for $Cl$ and $Cr$.
Since we have noticed that $Cl$ and $Cr$ are not proportional with the Thymio motors speed, we have decided to use the speed used to derive this coefficent as the constant speed input in our controller.


<div align="center">
    <img src="img/Thymio_wheel.jpeg" alt="Description of Thymio wheel" width="650" style="margin-right: 10px;"/>
    <img src="img/Cl_Cr.png" alt="Description of second image" width="400"/>
</div>



$$Cl = 69.33821285 \frac{s}{rad}Thymio units,Cr = 70.74580573 \frac{s}{rad}Thymio units$$



#### **Calcultating the covariance matrix of the motion model $Q_t$**


$$
Q_t = 
\begin{bmatrix}
Var(x_t) & Cov(x_t, y_t) & Cov(x_t, \gamma_t) \\
Cov(y_t, x_t) & Var(y_t) & Cov(y_t, \gamma_t) \\
Cov(\gamma_t, x_t) & Cov(\gamma_t, y_t) & Var(\gamma_t)
\end{bmatrix}
$$

To implement a Kalman filter, we first need to know the variance of the motion model. 
While my odometry system provides only the velocity of the two motors, we can use the geometry of the Thymio to calculate the position and orientation of the robot, hence: $$x=f_1(v_r,v_l) = \frac{R}{2} (v_r + v_l) \cos(\gamma) \Delta t$$ $$y=f_2(v_r,v_l) = \frac{R}{2} (v_r + v_l) \sin(\gamma) \Delta t$$ $$\gamma=f_3(v_r,v_l) =\frac{R}{L} (v_r-v_l) \Delta t$$ where $v_r$ and $v_l$ are the velocity of the two motors, $R$ is the radius of the wheel and $L$ is the distance between the two wheels . Now we have to take in account the error propagation, so we can calculate $$Var(x) = (\frac{\partial f_1}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_1}{\partial v_l})^2 Var(v_l) +2(\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) $$ $$Var(y) = (\frac{\partial f_2}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_2}{\partial v_l})^2 Var(v_l) +2(\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) $$ $$Var(\gamma) = (\frac{\partial f_3}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_3}{\partial v_l})^2 Var(v_l) +2 (\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l)  $$ 

We can consider $v_r$ and $v_l$ uncorrelated so after deriving the partial derivatives we can calculate the diagonal terms of $Q_t$ as follows:

$$Var(x) =  (\frac{R}{2} \cos(\gamma) \Delta t)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(y) = (\frac{R}{2} \sin(\gamma) \Delta t)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(\gamma) =  (\frac{R}{L}\Delta t)^2 (Var(v_r)+Var(v_l))$$

Next we need to calculate the off-diagonal terms of $Q_t$, we can do it again by taking in account the error propagation:
$$ Cov(x,y) = Cov(y,x)  = (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_2}{\partial v_r}) Var(v_r) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_2}{\partial v_l}) Var(v_l) + (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_2}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_2}{\partial v_r}) Cov(v_l,v_r)$$
$$ Cov(x,\gamma) = Cov(\gamma,x) = (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_3}{\partial v_r}) Var(v_r) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_3}{\partial v_l}) Var(v_l) + (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_3}{\partial v_r}) Cov(v_l,v_r)$$
$$ Cov(y,\gamma) = Cov(\gamma,y) = (\frac{\partial f_2}{\partial v_r}) (\frac{\partial f_3}{\partial v_r}) Var(v_r) + (\frac{\partial f_2}{\partial v_l}) (\frac{\partial f_3}{\partial v_l}) Var(v_l) + (\frac{\partial f_2}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_2}{\partial v_l}) (\frac{\partial f_3}{\partial v_r}) Cov(v_l,v_r)$$

After deriving the partial derivatives we obatain:

$$ Cov(x,y) = \frac{(R  \Delta t)}{4}^2(\cos(\gamma)\sin(\gamma))(Var(v_r)+Var(v_l)) $$
$$ Cov(x,\gamma) = \frac{(R \Delta t)^2}{2L} \cos(\gamma) (Var(v_r)-Var(v_l) )$$
$$ Cov(y,\gamma) = \frac{(R \Delta t)^2}{2L} \sin(\gamma) (Var(v_r)-Var(v_l)) $$

Now that we have our covariance matrix $Q_k$ formulation, we can calulate the parameters $Var(v_r)$ and $Var(v_l)$ by experimentation. By simply collecting data from our Thymio proceding at a constant speed of $200$ $Thymiounits$, we have obtained the variance of the two motors, in $(rad/s)^2$ 


<div align="center">
    <img src="img/motor_speed.png" alt="Description of the image" width="600"/>
</div>

$$Var(v_r) = 0.013997244458988999 (rad/s)^2$$
$$Var(v_l) = 0.007137898281767495 (rad/s)^2$$

After some simulation for different values of $\gamma$ from $0°$ to $180°$ we have found that the matrix $Q_t$ is diagonal.
<div align="center">
    <img src="img/Q.png" alt="Description of the image" width="600"/>
</div>

To ensure that the Kalman filter minimizes the variance of the error between the actual state and the predicted state, it is necessary to have a $Q$ matrix that is not time-varying ($Q_t = Q$). To achieve this, it is essential to find a $Q$ matrix that takes scalar inputs. Through simulation, we have selected a $Q$ matrix with values that maximize the variance values, thereby obtaining a conservative estimate of the matrix.

$$
Q =
\begin{bmatrix}
0.20000 & 0.00000 & 0.00000 \\
0.00000 & 0.20000 & 0.00000 \\
0.00000 & 0.00000 & 0.00070
\end{bmatrix}

$$

Another crucial aspect to consider is that in our model the input contribution ($ B_{t-1}u_{t-1}$) represents a deterministic quantity. As a result, our current $Q$ matrix accounts solely for the state error, excluding any input error. A common practical approach in control theory is to incorporate the input error into an experimental variable, $\alpha$, which requires experimental tuning. Consequently, our actual $Q$ matrix becomes $Q$ =$\alpha  Q$, effectively encompassing the error associated with the input as well. After some tuning we have found that a good matrix $Q$ for our application is:

$$
Q = 

\begin{bmatrix}
2 & 0 & 0. \\
0& 2 & 0 \\
0 & 0 & 0.07
\end{bmatrix}
$$


#### **Deriving the observation model of the Thymio Robot**

Source: https://automaticaddison.com/how-to-derive-the-observation-model-for-a-mobile-robot/

Now that we have a linear state space model of our Thyimio that includes also the process noise, we need an observation model in order to be able to implement the Kalman filter. 

An observation model describes how expected sensor outputs $y_t$ are related to the state vector $s_t$, considering also a vector of observation noise $ \nu_{t} = [ \nu^1_{t}, \nu^2_{t}, \nu^3_{t} ] ^T$
with zero mean and covariance matrix $R$ (which is computed after some experimentation explained in the next section). 
The observation model is defined as follows: 
$$y_t = H_t s_t + \nu_t$$ 
Since we are able to detect the position and the orientation of the Thyimio in the environment using camera, our matrix $H$ is the identity, hence we can define our observation model as follows:

$$
y_{t} =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}

\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
+
\begin{bmatrix}
\nu^1_{t}\\
\nu^2_{t}\\
\nu^3_{t}
\end{bmatrix}
$$

#### **Calcultating the covariance matrix of the observation model $R_t$**

The matrix $R_t$ is obtain by experimentation, and contain the variance of the observation noise, in our case the variance of the position and orientation of the Thymio in the environment, we can denote the observation vector as follows:

$$
z_{t} =
\begin{bmatrix}
x^o_{t} \\
y^o_{t} \\
\gamma^o_{t}
\end{bmatrix} 
$$

We can calculate the variance of the observation and it will be:

$$
R_t = 
\begin{bmatrix}
Var(x^o) & Cov(x^o, y^o) & Cov(x^o, \gamma^o) \\
Cov(y^o, x^o) & Var(y^o) & Cov(y^o, \gamma^o) \\
Cov(\gamma^o, x^o) & Cov(\gamma^o, y^o) & Var(\gamma^o)
\end{bmatrix}
$$
Note that those term are different from the ones of the motion model, because the observation noise is releted with our camera and our image vision approach, not with the Thymio itself. This matrix changes in relation with the visibility of our Thymio, for simplicity we can assume two different scenarios, when the Thymio is visible and when it is not, so we can define just 2 matrix $R$ and $R_{nc}$.If we can not detect it the incertence grows to infinity, so we can define a matrix $R_{nc}$, in this case we set the variance of the observation noise to infinity:
$$
R_{nc} = 
\begin{bmatrix}
\infty &0 & 0 \\
0 & \infty & 0 \\
0 & 0 & \infty 
\end{bmatrix}
$$

Now I can compute $R_t = R$ through experimentation, first we need to create an A4 paper as a calibration grid, ensuring the most accurate positioning of the Thymio robot before detecting its location and orientation. The A4 paper was easly generated using ChatGPT-4. The LLM after getting as input the shapes and the dimension needed for the grid was able to generate the following file:

<div align="center">
    <img src="img/square_design.png" alt="Description of the image" width="400"/>
</div>

Then we have computed the required variance mesuring the delta between the measure of the Thymio provided by the camera and the real position of the Thymio. We have repeated this 10 times and calculated our average values for $Var(x^o)$, $Var(y^o)$ and $Var(\gamma^o)$. To be consistent with the units of the motion model, we have converted the camera measure from pixels to millimiters.

<div align="center">
    <img src="img/R_exp.jpg" alt="Description of the image" width="600"/>
</div>




$$Var(x^o) = 0.7 (mm)^2$$
$$Var(y^o) = 0.7 (mm)^2$$
$$Var(\gamma^o) = 0.0014 (rad)^2$$

Hence our $R$ matrix is:
$$
R = 
\begin{bmatrix}
0.7 &0 & 0 \\
0 & 0.7 & 0 \\
0 & 0 & 0.0014
\end{bmatrix}
$$


#### **Implementing the Extended Kalman Filter**

Source: https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/

Since our model is not linear (the $B$ matrix has a trigonometrical relation with the state varaiable $\gamma$), we have to implement an Extended Kalman Filter

Due to the previous consideration we will have both the $R$ and the $R_{nc}$ to taking in to account the possibility of the Thymio being visible or not. We will use the $R_{nc}$ when the Thymio is not visible and $R$ when it is visible.

For the implementation we would refer to the measurements of the position and orientation of the Thymio in the environment obtained using the camera as 

$$
z_{t} =
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix} 
$$


In [ ]:
##############################################################################################################
## Code insipiration from Addison Sears-Collins
# https://automaticaddison.com
##############################################################################################################
import numpy as np

R = 20 #mm
L = 105 #mm
var_vr = 0.1 #da cambiare
var_vl = 0.1 #da cambiare
 
# State matrix A

A_k_minus_1 = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])
 
# Input matrix B
def getB(yaw, deltak):
    B = np.array([  [np.cos(yaw)*deltak, 0],[np.sin(yaw)*deltak, 0],[0, deltak]])
    return B

# Process noise w
process_noise_w_k_minus_1 = np.array([0,0,0])
     
# State model noise covariance matrix Q_k
q11 = 2 
q22 = 2
q33 = 0.07
Q_k = np.array([[q11, 0, 0],[0, q22, 0],[0, 0, q33]])                 
# Measurement matrix H_k
H_k = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])
                         
# Sensor measurement noise covariance matrix R_k
r11 = 0.7 #da cambiare
r22 = 0.7 #da cambiare
r33 = 0.0014 #da cambiare
R_k = np.array([[r11,0,0],[0,r22,0],[0,0,r33]])

# Sensor measurement noise covariance matrix R_k_nc
R_k_nc = np.array([[np.inf,np.inf,np.inf],[np.inf,np.inf,np.inf],[np.inf,np.inf,np.inf]])
                 
# Sensor noise v
sensor_noise_v_k = np.array([0,0,0])

# Initial state covariance matrix
# choose how to initialize it
P_k_minus_1 = np.array([[r11,0,0],[0,r22,0],[0,0,r33]]) 
                                               
                                               

#camera_vision = True means that the camera is used to estimate the position
#camera_vision = False means that the camera is covered so it is not used to estimate the position
 
def ekf(z_k_observation_vector, state_estimate_k_minus_1,control_vector_k_minus_1, P_k_minus_1, dk,camera_vision):

    # Prediction step

    #Prediction of the state estimate
    state_estimate_k = A_k_minus_1 @ (state_estimate_k_minus_1) + (getB(state_estimate_k_minus_1[2],dk)) @ (control_vector_k_minus_1) + (process_noise_w_k_minus_1)
    print(state_estimate_k)             
    # Predicton the state covariance estimate 
    yaw = state_estimate_k[2]
    P_k = A_k_minus_1 @ P_k_minus_1 @ A_k_minus_1.T + Q_k
         
    # Update step

    measurement_residual_y_k = z_k_observation_vector - ((H_k @ state_estimate_k) + (sensor_noise_v_k))
             
    # Calculate the measurement residual covariance
    if camera_vision == True:
        S_k = H_k @ P_k @ H_k.T + R_k
    else:
        S_k = H_k @ P_k @ H_k.T + R_k_nc
         
    # Calculate the near-optimal Kalman gain

    K_k = P_k @ H_k.T @ np.linalg.pinv(S_k)
         
    # Calculate an updated state estimate for time k
    state_estimate_k = state_estimate_k + (K_k @ measurement_residual_y_k)
    print(state_estimate_k) 
    # Update the state covariance estimate for time k
    P_k = P_k - (K_k @ H_k @ P_k)
     
    # Return the updated state and covariance estimates
    return state_estimate_k, P_k



#### **Control law RICONTROLLARE FORSE INUTILE SPIEGARE IL CODICE COSI' IN DETTAGLIO**

For the control law we have choose to use a simple PI controller, that works on the angle error between the desired angle and the current angle of the Thymio. The idea is the following: first it calculates the angle of the line between pointInitial and pointFinal, then determines the error between the desired angle of the line and the current angle of the robot. It uses proportional (P) and integral (I) control to calculate the differential speed needed to correct the angular error. Then it computes the perpendicular distance from the current position to the line between pointInitial and pointFinal and
implements proportional (P), control for correcting the position relative to the desired line. 


In [ ]:
def PIDcontrol(pointInitial,pointFinal,position,angle,dt,alignMode):

    #make controller for the angle
    angleOfLine = np.arctan2(pointFinal[1]-pointInitial[1],pointFinal[0]-pointInitial[0])# - np.pi


    if angleOfLine < 0:
        angleOfLine = angleOfLine + 2*np.pi
    #print(angleOfLine)

    angleError = angleOfLine-angle



    if angleError > np.pi:
        angleError -= 2*np.pi
    elif angleError < -np.pi:
        angleError += 2*np.pi


    Kangle = 50
    KIangle = 5

    differentialSpeed = Kangle*angleError + KIangle*angleError*dt

    distanceToLine = getDistance(position,pointInitial,pointFinal)

    #print("distance error:" + str(distanceToLine) + " angle error:" + str(angleError))

    Kdistance = 0.3
    KIdistance = 0.1
    KDdistance = 0.1
    

    posCorrection = Kdistance*distanceToLine

    constantSpeed = 200


    if alignMode == True:
        constantSpeed = 0
        posCorrection = 0
        posCorrection = 0

    l_speed = -differentialSpeed - posCorrection + constantSpeed
    r_speed = differentialSpeed + posCorrection + constantSpeed


    return l_speed, r_speed, distanceToLine, angleError

# **Main**

# **Conclusions**


#### Sources

Chat-GPT4: https://chat.openai.com/

Copilot: https://copilot.github.com/

Automatic Addison: https://automaticaddison.com

